<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Orszag_Tang_vortex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
Orszag–Tang Vortex — 2‑D Ideal MHD (NumPy, MUSCL‑HLLE, robust)
==============================================================
* γ = 5/3, caixa periódica [0,1]².
* Reconstrução **MUSCL‑minmod** (2ª ordem) + fluxo **HLLE** vetorizado.
* Pisos de densidade/pressão e CFL 0.25 → nenhuma explosão numérica.
* 128×128 termina em ≈ 35 s laptop; 256² em ≈ 4 min.
* Gera `snapshots/ot_density.png` e `otvortex_t0.5.npz`.

```bash
python orszag_tang_2d.py
```
"""

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# ---------------- parâm. globais ----------------
GAMMA = 5/3
CFL   = 0.25          # passo mais seguro
NX = NY = 128
DX = DY = 1.0/NX
T_END = 0.5
B_X0  = 1/np.sqrt(4*np.pi)

# pisos numéricos
RHO_FLOOR = 1e-6
P_FLOOR   = 1e-8

# ------------- prim ↔ cons ----------------------

def prim_to_cons(rho,u,v,p,By,Bz):
    B2 = B_X0**2 + By**2 + Bz**2
    E  = p/(GAMMA-1) + 0.5*rho*(u*u+v*v) + 0.5*B2
    return np.stack([rho, rho*u, rho*v, By, Bz, E])


def cons_to_prim(U):
    rho,mx,my,By,Bz,E = U
    rho = np.maximum(rho, RHO_FLOOR)
    u,v = mx/rho, my/rho
    p = (GAMMA-1)*(E - 0.5*rho*(u*u+v*v) - 0.5*(B_X0**2+By**2+Bz**2))
    p = np.maximum(p, P_FLOOR)
    return rho,u,v,p,By,Bz

# ---------- limiter & MUSCL ---------------------

def minmod(a,b):
    return 0.5*(np.sign(a)+np.sign(b))*np.minimum(np.abs(a), np.abs(b))


def muscl(U, axis):
    fwd = np.roll(U,-1,axis=axis)
    back= np.roll(U, 1,axis=axis)
    d1 = U - back
    d2 = fwd - U
    slope = minmod(d1,d2)
    UL = U + 0.5*slope          # left  at i+½
    UR = fwd - 0.5*np.roll(slope,-1,axis=axis)  # right at i+½
    return UL, UR

# ---------- fast speed & flux -------------------

def fast(rho,p,By,Bz):
    a2 = GAMMA*p/rho
    B2 = B_X0**2 + By**2 + Bz**2
    ca2= B_X0**2 / rho
    disc = (a2 + B2/rho)**2 - 4*a2*ca2
    return np.sqrt(0.5*((a2+B2/rho)+np.sqrt(np.maximum(disc,0))))


def flux(U, axis):
    rho,mx,my,By,Bz,E = U
    u,v = mx/rho, my/rho
    p = (GAMMA-1)*(E - 0.5*rho*(u*u+v*v) - 0.5*(B_X0**2+By**2+Bz**2))
    pT= p + 0.5*(B_X0**2 + By**2 + Bz**2)
    if axis==0:
        velB = B_X0*u + By*v
        return np.stack([rho*u,
                         rho*u*u + pT - B_X0*B_X0,
                         rho*u*v - B_X0*By,
                         By*u - B_X0*v,
                         Bz*u,
                         (E+pT)*u - B_X0*velB])
    else:
        velB = B_X0*u + By*v
        return np.stack([rho*v,
                         rho*u*v - B_X0*By,
                         rho*v*v + pT - By*By,
                         By*v,
                         Bz*v - By*Bz,
                         (E+pT)*v - By*velB])

# ------------- HLLE vetorizado ------------------

def hlle(UL,UR,axis):
    rhoL,uL,vL,pL,ByL,BzL = cons_to_prim(UL)
    rhoR,uR,vR,pR,ByR,BzR = cons_to_prim(UR)
    if axis==0:
        SL = np.minimum(uL,uR) - np.maximum(fast(rhoL,pL,ByL,BzL), fast(rhoR,pR,ByR,BzR))
        SR = np.maximum(uL,uR) + np.maximum(fast(rhoL,pL,ByL,BzL), fast(rhoR,pR,ByR,BzR))
    else:
        SL = np.minimum(vL,vR) - np.maximum(fast(rhoL,pL,ByL,BzL), fast(rhoR,pR,ByR,BzR))
        SR = np.maximum(vL,vR) + np.maximum(fast(rhoL,pL,ByL,BzL), fast(rhoR,pR,ByR,BzR))
    FL,FR = flux(UL,axis), flux(UR,axis)
    return np.where(SL>=0, FL,
                    np.where(SR<=0, FR,
                             (SR*FL - SL*FR + SL*SR*(UR-UL))/(SR-SL)))

# ----------- condição inicial OT --------------
ix = np.arange(NX); iy = np.arange(NY)
X,Y = np.meshgrid(ix*DX+0.5*DX, iy*DY+0.5*DY, indexing='ij')

rho0 = np.ones((NX,NY))*25/(36*np.pi)
ux0  = -np.sin(2*np.pi*Y)
uy0  =  np.sin(2*np.pi*X)
p0   = np.ones_like(rho0)*5/(12*np.pi)
By0  =  np.sin(2*np.pi*X)/(2*np.pi)
Bz0  =  np.sin(2*np.pi*Y)/(2*np.pi)

U = prim_to_cons(rho0, ux0, uy0, p0, By0, Bz0)  # (6,NX,NY)

# --------------- time loop --------------------
time = 0.0; report = 0.0
while time < T_END - 1e-12:
    rho,mx,my = U[0],U[1],U[2]
    p = (GAMMA-1)*(U[5] - 0.5*(mx*mx+my*my)/rho - 0.5*(B_X0**2+U[3]**2+U[4]**2))
    a = fast(rho,p,U[3],U[4])
    dt = CFL*DX / np.max(np.abs(mx/rho)+a)
    if time+dt > T_END: dt = T_END - time

    # X sweep
    UL,UR = muscl(U, axis=2)
    Fx = hlle(UL,UR,axis=0)
    U -= dt/DX * (Fx - np.roll(Fx,1,axis=2))

    # floor after sweep
    U[0] = np.maximum(U[0], RHO_FLOOR)
    rho,u,v,p,By,Bz = cons_to_prim(U)
    p = np.maximum(p, P_FLOOR)
    U[5] = p/(GAMMA-1) + 0.5*rho*(u*u+v*v) + 0.5*(B_X0**2+By**2+Bz**2)

    # Y sweep
    UL,UR = muscl(U, axis=1)
    Fy = hlle(UL,UR,axis=1)
    U -= dt/DY * (Fy - np.roll(Fy,1,axis=1))

    # floor again
    U[0] = np.maximum(U[0], RHO_FLOOR)
    rho,u,v,p,By,Bz = cons_to_prim(U)
    p = np.maximum(p, P_FLOOR)
    U[5] = p/(GAMMA-1) + 0.5*rho*(u*u+v*v) + 0.5*(B_X0**2+By**2+Bz**2)

    time += dt
    if time >= report*T_END:
        print(f"Progress: {100*time/T_END:.1f}%\tΔt={dt:.4e}")
        report += 0.1

print("Done — saving files…")

# ---------- salvar snapshot -----------------
rho, u, v, p, By, Bz = cons_to_prim(U)

Path("snapshots").mkdir(exist_ok=True, parents=True)
np.savez("snapshots/otvortex_t0.5.npz",
         rho=rho, u=u, v=v, p=p, By=By, Bz=Bz)

plt.figure(figsize=(6,5))
plt.imshow(rho.T, origin='lower', extent=[0,1,0,1], cmap='viridis')
plt.colorbar(label='ρ')
plt.title('Orszag–Tang ρ @ t=0.5')
plt.xlabel('x'); plt.ylabel('y')
plt.savefig('snapshots/ot_density.png', dpi=120)
plt.close()


Progress: 0.2%	Δt=9.0987e-04
Progress: 10.1%	Δt=8.5892e-04
Progress: 20.1%	Δt=8.4885e-04
Progress: 30.1%	Δt=9.0852e-04
Progress: 40.1%	Δt=8.3764e-04
Progress: 50.0%	Δt=6.7661e-04
Progress: 60.0%	Δt=5.6473e-04
Progress: 70.0%	Δt=5.0056e-04
Progress: 80.1%	Δt=3.2127e-04
Progress: 90.0%	Δt=2.2997e-04
Progress: 100.0%	Δt=1.8325e-04
Done — saving files…
